In [105]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.api import Api
import numpy as np
import asyncio
from geopy import distance
import drawsvg as draw
import typing

np.set_printoptions(suppress=True)

### CONSTANTS

In [106]:

LEFT, BOTTOM = 45.414580, 12.296868#50.050158, 19.931309#  kraków
RIGHT, TOP = 45.448136, 12.368594#50.066532, 19.948461 #kraków

ZOOM_LEVEL = 15

overpass = Overpass()

In [107]:
async def get_points_of_element(el):
    coordinates = el.geometry()["coordinates"]
    points = np.array(coordinates[0])

    return points

In [108]:
async def get_points_of_element(el):
    coordinates = el.geometry()["coordinates"]
    points = np.array(coordinates).reshape(-1, 2)

    return points

In [109]:
def draw_path(points, **options):
    
    points = points.flatten().tolist() #flatten to list of numbers [x1, y1, x2,y2...]
    l = draw.Lines(*points, #unpack list to separate arguments
                 **options)
    return l

In [110]:
def project_web_mercator(points): #numpy array of point degrees
    # print(f"In degrees {points = }")
    points = np.deg2rad(points)

    lats = points[:, 1]
    lons = points[:, 0]
    # print(f"In radians {lats = }")
    # print(f"In radians {lons = }")

    projection_constant = np.divide(256, 2*np.pi) * np.exp2(ZOOM_LEVEL)
   
    xs = projection_constant * (lons + np.pi)


    # print(f"In radians {lats = }")
    ys = projection_constant * (np.pi - np.log(np.tan((np.pi/4) +  (lats/2))))


    return np.column_stack((xs, ys))



In [111]:
def get_center_projection(bbox):
    left, bottom, right, top = bbox
    center = (bottom+top)/2, (left+right)/2
    center_point = np.array(center).reshape(1, 2)
    return project_web_mercator(center_point)


In [112]:
BBOX = [LEFT, BOTTOM, RIGHT, TOP]
PROJECTED_CENTER = get_center_projection(BBOX)


async def plot_ways(drawing, selector, **kwargs): #for each query
    query = overpassQueryBuilder(bbox=BBOX, elementType=['way'], selector=selector)
    elements = overpass.query(query).elements()
    print(f"Got {len(elements)} ways for this query.")
    #RETRIEVE POINTS
    async with asyncio.TaskGroup() as tg:
        tasks = [tg.create_task(get_points_of_element(el)) for el in elements]
    
    list_of_ways = [task.result() for task in tasks]

    for way in list_of_ways:
        road_projection = project_web_mercator(way)
        sim_road = road_projection-PROJECTED_CENTER
        drawing.append(draw_path(sim_road, **kwargs))




In [ ]:
async def plot_ways(drawing, selector, **kwargs): #for each query
    query = overpassQueryBuilder(bbox=BBOX, elementType=['way'], selector=selector)
    elements = overpass.query(query).elements()
    print(f"Got {len(elements)} ways for this query.")
    #RETRIEVE POINTS
    async with asyncio.TaskGroup() as tg:
        tasks = [tg.create_task(get_points_of_element(el)) for el in elements]
    
    list_of_ways = [task.result() for task in tasks]

    for way in list_of_ways:
        road_projection = project_web_mercator(way)
        sim_road = road_projection-PROJECTED_CENTER
        drawing.append(draw_path(sim_road, **kwargs))


### LINE OPTIONS

In [113]:

black_line = {
    "fill": "none",
    "stroke": "black",
    "close": False,
}

blue_area = {
    "fill": "blue",
    "stroke": "none",
    "close": True,
}
green_line = {
    "fill": "none",
    "stroke": "green",
    "close": False,
}
green_area = {
    "fill": "green",
    "stroke": "none",
    "close": True
}

# MAIN

### SELECTORS

In [114]:
ROADS_SELECTOR = '"highway"~"^(((motorway|trunk|primary|secondary|tertiary)(_link)?)|unclassified|residential|living_street|pedestrian|service|track)$"'


### DRAW

In [115]:
d = draw.Drawing(1000, 1000, origin='center')
r = draw.Rectangle(-500, -500, 1000, 1000, fill="#f3de67")
d.append(r)


BBOX = [LEFT, BOTTOM, RIGHT, TOP]


await plot_ways(d, ROADS_SELECTOR, **black_line)
await plot_ways(d, "'natural'='water'", **blue_area)
await plot_ways(d, "'landuse'='allotments'", **green_line)
await plot_ways(d, "'leisure'='park'", **green_area)

Got 5748 ways for this query.
Got 177 ways for this query.
Got 0 ways for this query.
Got 45 ways for this query.


### RENDER

In [116]:
d.set_pixel_scale(1)
d.save_png('example.png')
d.save_svg('example.svg')